In [23]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter, deque
from functools import lru_cache
import numba
from itertools import permutations, combinations, product

In [24]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [25]:
def mod(input):
    _ = input.split("..")
    b = int(_[-1])
    a = int(_[0].split("=")[-1])
    return (a,b)

mod("x=-12..41")

(-12, 41)

In [26]:
parsedInput = [[mod(xyz) for xyz in t] for t in [[t[0].split(" ")[-1],t[1].split(" ")[-1],t[2].split(" ")[-1]] for t in [line.split(",") for line in rawstring.splitlines()]]]

In [27]:
onOff = [True if 'on' in line else False for line in rawstring.splitlines()]

In [63]:
cuboid = np.zeros(shape=(101,101,101),dtype=bool)
for xyz,t in zip(parsedInput[0:2],onOff[0:2]):
    x,y,z = xyz
    print(np.sum(cuboid >= 1))
    print(x,y,z)
    cuboid[
        np.clip(x[0]+50,0,100):np.clip(x[1]+51,0,100),
        np.clip(y[0]+50,0,100):np.clip(y[1]+51,0,100),
        np.clip(z[0]+50,0,100):np.clip(z[1]+51,0,100)] = t

0
(-12, 41) (-1, 48) (-27, 19)
126900
(-40, 7) (-47, 2) (-24, 22)


In [64]:
np.sum(cuboid)

236180

In [29]:
list(product((-12, 41),(-1, 48),(-27, 19)))

[(-12, -1, -27),
 (-12, -1, 19),
 (-12, 48, -27),
 (-12, 48, 19),
 (41, -1, -27),
 (41, -1, 19),
 (41, 48, -27),
 (41, 48, 19)]

In [30]:
from graphviz import Digraph

In [31]:
gra = Digraph()

In [133]:
class Cube():
    def __init__(self, slice,) -> None:
        self.xrange = (slice[0][0], slice[0][1])
        self.yrange = (slice[1][0], slice[1][1])
        self.zrange = (slice[2][0], slice[2][1])
        self.state = 0
        self.slice = slice

    def volume(self):
        return np.prod([self.xrange[1]-self.xrange[0]+1, self.yrange[1]-self.yrange[0]+1, self.zrange[1]-self.zrange[0]+1])
    
    def vertices(self):
        return list(product(*self.slice))

    def overlaps(self, other, return_count=False):
        other_vertices = other.vertices()
        overlapping = 0
        for vertex in other_vertices:
            if (self.xrange[0] <= vertex[0] <= self.xrange[1]) and (self.yrange[0] <= vertex[1] <= self.yrange[1]) and (self.zrange[0] <= vertex[2] <= self.zrange[1]):
                overlapping += 1
        if return_count:
            return overlapping >= 1, overlapping
        return overlapping >= 1

    def intersect(self, other):
        resulting_cubes = []
        if self.overlaps(other, return_count=True)[1] == 8:
            return [other]
        if not self.overlaps(other):
            return [other]
            
        if other.xrange[0] < self.xrange[0] < other.xrange[1]:
            resulting_cubes.append(Cube([(other.xrange[0],self.xrange[0]),other.yrange,other.zrange]))
            resulting_cubes.append(Cube([(self.xrange[0]+1,other.xrange[1]),other.yrange,other.zrange]))
        elif other.yrange[0] < self.yrange[0] < other.yrange[1]:
            resulting_cubes.append(Cube([other.xrange,(other.yrange[0],self.yrange[0]),other.zrange]))
            resulting_cubes.append(Cube([other.xrange,(self.yrange[0]+1,other.yrange[1]),other.zrange]))
        elif other.zrange[0] < self.zrange[0] < other.zrange[1]:
            resulting_cubes.append(Cube([other.xrange,other.yrange,(other.zrange[0],self.zrange[0])]))
            resulting_cubes.append(Cube([other.xrange,other.yrange,(self.zrange[0]+1,other.zrange[1])]))
        elif other.xrange[0] < self.xrange[1] < other.xrange[1]:
            resulting_cubes.append(Cube([(other.xrange[0],self.xrange[1]),other.yrange,other.zrange]))
            resulting_cubes.append(Cube([(self.xrange[1]+1,other.xrange[1]),other.yrange,other.zrange]))
        elif other.yrange[0] < self.yrange[1] < other.yrange[1]:
            resulting_cubes.append(Cube([other.xrange,(other.yrange[0],self.yrange[1]),other.zrange]))
            resulting_cubes.append(Cube([other.xrange,(self.yrange[1]+1,other.yrange[1]),other.zrange]))
        elif other.zrange[0] < self.zrange[1] < other.zrange[1]:
            resulting_cubes.append(Cube([other.xrange,other.yrange,(other.zrange[0],self.zrange[1])]))
            resulting_cubes.append(Cube([other.xrange,other.yrange,(self.zrange[1]+1,other.zrange[1])]))

        if len(resulting_cubes)==0:
            return [other]
        assert(sum([cube.volume() for cube in resulting_cubes]) == other.volume())

        retVal = []
        for o in resulting_cubes:
            for e in self.intersect(o):
                if isinstance(e,Cube):
                    retVal.append(e)
        return retVal


In [134]:
c0 = Cube(parsedInput[0])
c1 = Cube(parsedInput[1])
print(f"{c0.volume()=}")
print(f"{c1.volume()=}")
print(f"{c0.volume()+c1.volume()=}")

c0.volume()=126900
c1.volume()=112800
c0.volume()+c1.volume()=239700


In [135]:
sum([x.volume() for x in c1.intersect(c0)])

126900

In [136]:
sum([x.volume() for x in c1.intersect(c0)])

126900

In [137]:
[c1.overlaps(x,return_count=True) for x in c1.intersect(c0)]

[(False, 0), (True, 8), (True, 4), (True, 4), (True, 2)]

In [138]:
[x.slice for x in c1.intersect(c0)]

[[(-12, 41), (-1, 48), (-27, -25)],
 [(-12, 6), (-1, 1), (-24, 19)],
 [(-12, 6), (2, 48), (-24, 19)],
 [(7, 41), (-1, 1), (-24, 19)],
 [(7, 41), (2, 48), (-24, 19)]]

In [139]:
c1.slice

[(-40, 7), (-47, 2), (-24, 22)]

In [124]:
3680-320

3360

In [121]:
239700-236180

3520

In [117]:
np.sum(cuboid)

236180

In [98]:
sum([c.state*c.volume() for c in cubes])

112800

In [97]:
nodes = []
edges = []
vols = []

cubes = []

for idx,args in enumerate(zip(parsedInput[0:2],onOff[0:2])):
    print(idx)
    t,s = args
    cCube=Cube(t)
    cCube.state = s

    newCubes = []
    newCubes.append(cCube)
    for c in cubes:
        if cCube.overlaps(c):
            for nc in c.intersect(cCube):
                if not nc.overlaps(cCube):
                    newCubes.append(nc)
        else:
            newCubes.append(c)

    cubes = newCubes
    for c in cubes:
        if c != cCube:
            if cCube.overlaps(c):
                c.state = s

0
1


In [36]:
sorted(vols)

[]

In [37]:
for x in nodes:
    gra.node(*x)

In [38]:
len(edges)

0

In [39]:
for x,y,c in edges:
    gra.edge(str(x),str(y),color=c)

In [40]:
gra

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [ ]:
gra.render()

'Digraph.gv.pdf'